#Chapitre 5  

Dans ce TP, nous allons utiliser le LLM [Mistral-7B-Instruct](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) pour effectuer une tâche de classification de sentiment.  

Le modèle a 7 Milliards de paramètres (7B) et il est finetuné sur un dataset d'instructions (Instruct)  

Le modèle étant de grande taille, nous allons d'abord le quantizer afin de pouvoir l'exécuter.  
Voici les outils que nous allons utiliser:  
- llama.cpp: afin de quantizer le modèle  
- llama-cpp-python: un wrapper Python autour de llama.cpp  
- langchain: afin d'instancier et d'utiliser le LLM. Possède une classe LlamaCpp qui est un wrapper autour de llama-cpp-python  


Commençons par installer llama-cpp-python  et les packages dont on aura besoin

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5811182 sha256=ab6a8f7556e3b2971978d603281eb83391003dc068751eb2024e7fac33b18dc7
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 38955, done.
remote: Counting objects: 100% (7486/7486), done.
remote: Compressing objects: 100% (360/360), done.
remote: Total 38955 (delta 7311), reused 7126 (delta 7126), pack-reused 31469 (from 1)
Receiving objects: 100% (38955/38955), 62.73 MiB | 16.17 MiB/s, done.
Resolving deltas: 100% (28356/28356), done.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [ ]:
pip install huggingface_hub langchain

Téléchargeons le modèle à partir du hub huggingface. La fonction snapshot_download sert à sauvegarder le modèle localement (sur le disque de la session Colab).  
En l'occurrence, il sera sauvegardé dans le répertoire mistral-hf  

In [ ]:
from huggingface_hub import snapshot_download
from huggingface_hub import login
model_id = "Qwen/Qwen2.5-Coder-7B-Instruct"
snapshot_download(repo_id=model_id, local_dir="qwen-coder", local_dir_use_symlinks=False, revision="main")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/6.16k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

'/content/qwen-coder'

Utilisons le script convert.py de llama.cpp afin de convertir le modèle vers un format utilisable par la library (gguf).  
Durant la conversion, on peut spécifier le degré de quantization, entre autres paramètres.  
Voici quelques valeurs possibles pour le paramètre de quantization **outtype**:  
- f16  
- q8_0  
- q4_0  
- ...  

In [ ]:
!ls /content/qwen-7B-Coder-Instruct.gguf




/content/qwen-7B-Coder-Instruct.gguf


In [ ]:
!python llama.cpp/convert_hf_to_gguf.py \
  /content/qwen-coder \
  --outfile /content/qwen-7B-Coder-Instruct.gguf \
  --outtype f16



INFO:hf-to-gguf:Loading model: qwen-coder
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,         torch.bfloat16 --> F16, shape = {3584, 152064}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.bfloat16 --> F32, shape = {3584}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.bfloat16 --> F16, shape = {18944, 3584}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.bfloat16 --> F16, shape = {3584, 18944}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.bfloat16 --> F16, shape = {3584, 18944}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.bfloat16 --> F32, shape = {3584}
INFO:hf-to-gguf:blk.0.attn_k.bias,         torch.bfloat16 --> F32, shape = {512}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.bfloat16 --> F16, shape = {3584, 512}
INFO:h

Chargons le modèle sur le GPU

In [ ]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
  model_path="/content/qwen-7B-Coder-Instruct.gguf",  # Chemin vers le modèle converti
  temperature=0.,
  max_tokens=200,
  top_p=1,
  n_gpu_layers=100,    # Nombre de couches à charger sur le GPU
  verbose=True
)


ValidationError: 1 validation error for LlamaCpp
  Value error, Could not load Llama model from path: /content/qwen-7B-Coder-Instruct.gguf. Received error  [type=value_error, input_value={'model_path': '/content/...: None, 'grammar': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

Testons le modèle en inférence  

In [ ]:
response = llm(
"""
This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //
"""
)
print(response)

NameError: name 'llm' is not defined

Testez quelques exemples de prompts pour apprécier les capacités et limites du modèle (Génération libre, Zero-Shot, Few-shot, CoT, ...).  

Vous pourrez trouver quelques exemples ici :  
- [Prompt Engineering guide](https://www.promptingguide.ai/)  
- [Prompt Engineering](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/)

Quel est le résultat de votre analyse ?  


Réponse : [à compléter]

# Classification  
Nous allons à présent utiliser le LLM [Flan-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) et le comparer au modèle Mistral-7B-Instruct sur une tâche de classification de sentiment.   

Il existe des exemples d'utilisation du modèle sur le lien ci-dessus. Vous êtes libre de choisir la taille du modèle que vous souhaitez.  

Vous pouvez commencer par le modèle que vous souhaitez (il n'y aura probablement pas assez d'espace sur GPU pour tester les deux en même temps)    

Nous allons utiliser le dataset [SST2](https://huggingface.co/datasets/glue/viewer/sst2) du benchmark GLUE.  

Téléchargez le dataset SST2

In [1]:
!pip install datasets
!pip install transformers
!pip install langchain langchain-community
!pip install langchain
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.1 MB/s eta 0:00:00
   ━━━

In [31]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq

dataset = load_dataset("glue", "sst2")


print(dataset)
print(dataset["train"][0])



model_name = "google/flan-t5-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


prompt = "Classify the sentiment of the following sentence as positive or negative:\nSentence: 'Je suis tres triste'"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(inputs.input_ids, max_new_tokens=5)
print("Output:", tokenizer.decode(outputs[0], skip_special_tokens=True))


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Output: negative


Etant donné que le LLM peut avoir un temps d'inférence important. Echantillonnez un sous-ensemble d'exemples du dataset SST2 sur lequel vous effectuerez la classification (à déterminer selon la vitesse d'inférence).

In [32]:
import random

# Convertir en liste pour l'échantillonnage
train_list = list(dataset["train"])
test_list = list(dataset["test"])

# Définir la taille de l'échantillon
subset_size = 100  # Ajustez selon la vitesse d'inférence

# Échantillonnage aléatoire du sous-ensemble
random.seed(42)  # Assurez la reproductibilité
train_subset = random.sample(train_list, subset_size)
test_subset = random.sample(test_list, subset_size)

# Afficher un exemple pour vérifier
print("Exemple d'entrée :", test_subset[0])

Exemple d'entrée : {'sentence': '... a weak , manipulative , pencil-thin story that is miraculously able to entertain anyway .', 'label': -1, 'idx': 1221}


Définissez le modèle de prompt que vous allez donner en entrée aux LLMs.  

Conseil: Vous pouvez utiliser la classe [PromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) de langchain qui permet de définir des modèles de prompts

In [33]:
# Exemple de phrase à analyser
sentence = test_subset[0]

# Construire le prompt
prompt = f"Analysez la phrase suivante et déterminez si le sentiment est positif ou négatif.\n\nPhrase : \"{sentence}\"\nSentiment :"

# Tokeniser le prompt
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

# Générer une réponse
outputs = model.generate(
    inputs.input_ids,
    max_length=50,  # Longueur maximale de la réponse
    num_beams=5,    # Beam search pour des réponses plus précises
    temperature=0.7
)

# Décoder la réponse
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Phrase : {sentence}\nRéponse du modèle : {response}")


Phrase : {'sentence': '... a weak , manipulative , pencil-thin story that is miraculously able to entertain anyway .', 'label': -1, 'idx': 1221}
Réponse du modèle : négatif


Définissez les fonctions de post-processing qui vont extraire la réponse des LLMs.  

- Attention, la fonction doit pouvoir gérer certains cas d'hallucination du modèle et à minima sortir une réponse "par défaut" (classe de sentiment neutre par exemple) si le post-processing n'est pas possible. Sinon, il ya un risque que votre fonction crash ou engendre le crash de l'inférence.    
- Testez votre fonction avec certaines sorties du LLM pour vous assurer qu'elle Fonctionne bien.  

In [34]:
def post_process_response(response):
    # Si la réponse est une liste, récupérez le texte généré dans le premier élément
    if isinstance(response, list):
        response_text = response[0]["generated_text"].strip().lower()
    else:
        # Si la réponse est directement une chaîne de caractères, on la traite directement
        response_text = response.strip().lower()

    # Vérification
    if "positif" in response_text:
        return "positif"
    elif "négatif" in response_text:
        return "négatif"
    else:
        # Cas où la réponse est non pertinente ou vide
        return "neutre"

# Réponses générées par le LLM
responses = [
    "Positif",
    "Le sentiment est négatif.",
    "Je pense que c'est clairement positif.",
    "Sentiment : négatif",
    "C'est difficile à dire, mais cela semble neutre.",
    "Lorem ipsum dolor sit amet",
    "",
]

sentiment = post_process_response(response)
print(f"Phrase : {sentence}\nRéponse du modèle : {response}\nSentiment : {sentiment}")


Phrase : {'sentence': '... a weak , manipulative , pencil-thin story that is miraculously able to entertain anyway .', 'label': -1, 'idx': 1221}
Réponse du modèle : négatif
Sentiment : neutre


Lancez l'évaluation de vos modèles sur le sous-ensemble du dataset.  

Note: Il est possible que vous ayez du mal à monter les performances du modèle Mistral au dela d'un certain point car:  
- Il est dans la fourchette basse en terme de taille de modèles  
- La quantization engendre une perte de performances  

In [35]:
from langchain.prompts import PromptTemplate
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
def evaluate_model(llm, data_subset, prompt_template):
    """
    Évalue un modèle LLM sur un sous-ensemble de données.
    """
    predictions = []
    ground_truth = []

    for example in data_subset:
        # Préparer le prompt
        prompt = prompt_template.format(sentence=example["sentence"])

        # Tokeniser le prompt
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

        # Générer la réponse avec le modèle
        outputs = llm.generate(
                  inputs.input_ids,
                  max_length=100,
                  num_beams=7,
                  temperature=0.7,
                  top_k=50,
                  top_p=0.9,
                  repetition_penalty=1.2,
              )

        # Décoder la réponse
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Post-traiter la réponse
        sentiment = post_process_response(response)

        # Collecter les prédictions et les vérités terrain
        ground_truth.append("positif" if example["label"] == 1 else "négatif")
        predictions.append(sentiment)

    print(ground_truth)
    print(predictions)
    accuracy = accuracy_score(ground_truth, predictions)
    return accuracy

prompt_template = PromptTemplate(
    input_variables=["sentence"],
    template="Analysez la phrase suivante et attribuez-lui un sentiment clair : 'positif', 'négatif' ou 'neutre'.\n\nPhrase : \"{sentence}\"\nSentiment :"
)

# Évaluation du modèle Flan-T5
print("Évaluation de Flan-T5...")
accuracy_flan = evaluate_model(model, test_subset, prompt_template)
print(f"Accuracy Flan-T5 : {accuracy_flan:.4f}")



Évaluation de Flan-T5...
['négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négat

Modifiez votre prompt afin d'améliorer les performances et relancez l'évaluation.

Par exemple, vous pouvez tester les modèles sur du 0-shot, 1-Shot, et Few-Shot learning pour voir s'il y a une différence dans les perfs.

Vous pouvez noter vos résultats intermédiaires au fur et à mesure ci-dessous.  

In [36]:
prompt_template_zero_shot = PromptTemplate(
    input_variables=["sentence"],
    template="Analysez la phrase suivante et attribuez-lui un sentiment clair : 'positif', 'négatif' ou 'neutre'.\n\nPhrase : \"{sentence}\"\nSentiment :"
)
prompt_template_one_shot = PromptTemplate(
    input_variables=["sentence"],
    template="""
    Voici un exemple de phrase et son sentiment :
    Phrase : "The movie was great, I loved it!"
    Sentiment : positif

    Analysez la phrase suivante et attribuez-lui un sentiment clair : 'positif', 'négatif' ou 'neutre'.
    Phrase : "{sentence}"
    Sentiment :
    """
)
prompt_template_few_shot = PromptTemplate(
    input_variables=["sentence"],
    template="""
    Voici quelques exemples de phrases et leurs sentiments :
    Phrase : "The movie was great, I loved it!"
    Sentiment : positif

    Phrase : "The film was boring and I didn't enjoy it."
    Sentiment : négatif

    Phrase : "It was an okay movie, not too good but not bad either."
    Sentiment : neutre

    Analysez la phrase suivante et attribuez-lui un sentiment clair : 'positif', 'négatif' ou 'neutre'.
    Phrase : "{sentence}"
    Sentiment :
    """
)
# Tester les trois approches
print("Évaluation Zero-Shot...")
accuracy_zero_shot = evaluate_model(model, test_subset, prompt_template_zero_shot)
print(f"Accuracy Zero-Shot : {accuracy_zero_shot:.4f}")

print("Évaluation One-Shot...")
accuracy_one_shot = evaluate_model(model, test_subset, prompt_template_one_shot)
print(f"Accuracy One-Shot : {accuracy_one_shot:.4f}")

print("Évaluation Few-Shot...")
accuracy_few_shot = evaluate_model(model, test_subset, prompt_template_few_shot)
print(f"Accuracy Few-Shot : {accuracy_few_shot:.4f}")

Évaluation Zero-Shot...
['négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négatif', 'négati

##experiment_1  
template_1 = Flan-T5-large
resultats_1: 26%
##experiment_2  
...

Si vous le souhaitez, vous pouvez reporter vos résultats sous forme de tableau avec cette syntaxe:  



| (Accuracy)   | Mistral-7B-Instruct |  Flan-T5-base |
|--------------|:-------------------:|--------------:|
| Experiment_1 |  33%                | 44%           |
| Experiment_2 |  88%                | 64%           |
| ...          | ...                 |    ...        |